In [ ]:
from aocd import get_data
import re
raw_data = get_data(day=10, year=2025).splitlines()
indicators = [{i for i, c in enumerate(m) if c == '#'} for text in raw_data for m in re.findall(r'\[([.#]+)\]', text)]
button_groups = [[set(map(int, b.split(','))) for b in re.findall(r'\(([^)]+)\)', line)] for line in raw_data]
joltages = [list(map(int, re.search(r'\{([^}]+)\}', line).group(1).split(','))) for line in raw_data]

## Part 1 - Bitmask Enumeration

In [ ]:
sum_presses = 0
for machine, (indicator, buttons) in enumerate(zip(indicators, button_groups)):

    num_buttons = len(buttons)
    min_presses = float('inf')
    
    # Try all 2^n combinations of buttons (each bit = one button on/off)
    for n in range((2**num_buttons)):
        pressed = set() # Which lights end up toggled
        presses = n.bit_count() # Count 1-bits = number of buttons pressed
        
        # Skip if this combination can't beat our best
        if presses >= min_presses:
            continue
        
        # Check each bit to see which buttons are "selected"
        for i in range(num_buttons):
            if n & (1 << i): # If bit i is set, press button i
                pressed = pressed ^ buttons[i] # XOR toggles the lights
        
        # Does this combo work? 
        if pressed == indicator:
            min_presses = min(min_presses, presses)
            if min_presses == 1: # Can't do better
                break

    sum_presses += min_presses
print(sum_presses)

494


## Part 2 - Integer Linear Programming

In [ ]:
import numpy as np
from scipy.optimize import milp, Bounds, LinearConstraint

# A_eq @ x = b_eq
# x = vector of press counts (one per button)
# b_eq = target joltages
# A_eq = matrix where A_eq[i][j] = 1 if button j affects counter i, else 0
# c: the cost vector (what are you minimizing per button press?)
# To minimize c @ x

min_presses = 0
for buttons, joltage in zip(button_groups, joltages):

    A_i = len(joltage)
    A_j = len(buttons)
    A = np.zeros((A_i, A_j))

    # Which button j affects which counter i
    for j, button in enumerate(buttons): 
        for i in button:
            A[i, j] = 1

    c = np.ones(A_j) # cost vector, what we are minimizing per button press, which is 1 for everything
    integrality = np.ones(A_j) # which values need to be integers, here all of them.
    constraints = LinearConstraint(A, joltage, joltage) # set both times to joltage to have an equality condition! 
    bounds = Bounds(lb=0, ub=max(joltage)) # each button is pressed >= 0 times and can't be pressed more than the max joltage
    res = milp(c=c, bounds=bounds, constraints=constraints, integrality=integrality)
    min_presses += sum(res.x)
print(min_presses)

19235.0


np.float64(49.0)